In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
def beta(point_type, k, l, x):
    if point_type == 1:
        return 0
    
    if point_type == 2:
        if k == 1:
            if l == 1:
                return 0.5
        if k == 0:
            if l == 2:
                return 0.5      

In [3]:
def a(v):
    l = np.sum(np.abs(v))
    if l == 1:
        return 0.5
    if l == 2:
        return 0.25

In [4]:
def get_elements_probas(p, point_tree_dict, mu, kappa, beta, a):
    elements = [
        'die',
        '(1, 1)',
        '(0, 2)',
        'x + (0, 1)',
        'x + (1, 0)',
        'x + (1, 1)',
        'x + (2, 0)',
        'x + (0, 2)',
        'x + (0, -1)',
        'x + (-1, 0)',
        'x + (-1, -1)',
        'x + (-2, 0)',
        'x + (0, -2)'
    ]

    probas = [
        mu[p.point_type],
        beta(p, 1, 1, point_tree_dict),
        beta(p, 0, 2, point_tree_dict),
        a([0, 1]),
        a([1, 0]),
        a([1, 1]),
        a([2, 0]),
        a([0, 2]),
        a([0, -1]),
        a([-1, 0]),
        a([-1, -1]),
        a([-2, 0]),
        a([0, -2])
    ]
    
    return elements, probas / np.sum(probas)

In [5]:
class Point():
    def __init__(self, point_type, x, t_1, t_2):
        self.point_type = point_type
        self.x = x
        self.t_1 = t_1
        self.t_2 = t_2
        
    def shifted(self, v, dt):
        v = np.array(v)
        return Point(self.point_type, self.x + v, self.t_2, self.t_2 + dt)
    
    def __str__(self):
        return '{} {} {} {} {}'.format(self.point_type, self.x[0], self.x[1], self.t_1, self.t_2)

In [6]:
def init_points_2d(size):
    R = []
    
    for i in range(-size, size, 1):
        for j in range(-size, size, 1):
            R.append(Point(1, np.array([i, j]), 0, 0))
            R.append(Point(2, np.array([i, j]), 0, 0))              
            
    return R

In [21]:
def get_x_str(x):
    return str((int(x[0]), int(x[1])))

def append_to_point_tree_dict(point_tree_dict, p):
    x_str = get_x_str(p.x)
    if x_str not in point_tree_dict:
        point_tree_dict[x_str] = [IntervalTree(), IntervalTree()]
    point_tree_dict[x_str][p.point_type - 1].append(Interval(p.t_1, p.t_2, 1))


def append_points(R, k, l, p, dt, point_tree_dict, noise_std):
    for i in range(k):
        coord_noise = np.random.normal(0, noise_std, 2)
        p = Point(1, p.x + coord_noise, p.t_2, p.t_2 + dt)
        R.append(p)
        append_to_point_tree_dict(point_tree_dict, p)
        
    for i in range(l):
        coord_noise = np.random.normal(0, noise_std, 2)
        p = Point(2, p.x + coord_noise, p.t_2, p.t_2 + dt)
        R.append(p)
        append_to_point_tree_dict(point_tree_dict, p)


def step(R, H, dt, idx, point_tree_dict, mu, kappa, beta, a, noise_std):
    p = R[idx]
    
    elements, probas = get_elements_probas(p, point_tree_dict, mu, kappa, beta, a)
    
    action = np.random.choice(a=elements, p=probas, size=1)[0]
    
    coord_noise = np.random.normal(0, noise_std, 2)
    
    if action[0:3] == 'x +':
        shift = [int(s) for s in action[4:].replace(')', '').replace('(', '').split(', ')]

        shifted = p.shifted(shift + coord_noise, dt)
        R.append(shifted)
        append_to_point_tree_dict(point_tree_dict, shifted)
        
    elif action[0] == '(':
        num_childs = [int(s) for s in action.replace(')', '').replace('(', '').split(', ')]
        
        append_points(R, num_childs[0], num_childs[1], p, dt, point_tree_dict, noise_std)
    
        
    H.append(p)
    del R[idx]

In [22]:
def simulate(R, H, T, mu, kappa, beta, a, noise_std, max_steps=10000):
    k = 0

    point_tree_dict = {}

    for x, point_type in [(str((int(p.x[0]), int(p.x[1]))), p.point_type) for p in R]:
        if x in point_tree_dict:
            point_tree_dict[x][point_type - 1].append(Interval(0, 0.0001, 1))
        else:
            point_tree_dict[x] = [IntervalTree(), IntervalTree()]
    
    pbar = tqdm(total=max_steps, position=0, leave=True)
    while k < max_steps:
        i = 0
        idx = 0
        found = 0
        
        while 1:
            if len(R) == 0:
                break

            idx = np.random.randint(len(R))

            if R[idx].t_2 < T:
                found = 1
                break
            if i >= len(R):
                break
            
            i += 1
        
        if len(R) == 0:
            return
        
        if found != 1:
            return
        
        dt = np.random.exponential()
        step(R, H, dt, idx, point_tree_dict, mu, kappa, beta, a, noise_std)
        
        k += 1
        pbar.update(1)

    return R, H

In [23]:
def get_max_t(all_points):
    max_t = 0
    for p in all_points:
        if p.t_2 > max_t:
            max_t = p.t_2
    
    return max_t

def get_min_coords(all_points):
    min_x = min_y = max_x = max_y = 0
    for point in all_points:
        if point.x[0] < min_x:
            min_x = point.x[0]
        if point.x[1] < min_y:
            min_y = point.x[1]
        if point.x[0] > max_x:
            max_x = point.x[0]
        if point.x[1] > max_y:
            max_y = point.x[1]

    return (min_x, min_y, max_x, max_y)

In [24]:
from intervaltree import Interval, IntervalTree

def build_interval_tree(points):
    interval_tree = IntervalTree()
    
    for i in tqdm(range(len(points)), position=0, leave=True):
        p = points[i]
        if p.t_1 + p.t_2 > 0:
            interval_tree.add(Interval(p.t_1, p.t_2, i))

    return interval_tree

In [25]:
def plot_points(t, points, interval_tree, xlim, ylim):
    coords = [[], []]
    
    idx = list(map(lambda i: i.data, interval_tree.at(t)))
    
    points_t = points[idx]
    
    
    color_dict = {
        1: 'C0',
        2: 'C1'
    }
    
    label_dict = {
        'C0': 'Men',
        'C1': 'Women'
    }
    
    x_y = np.array(list(map(lambda p: p.x, points_t)))
    
    labels = np.array(list(map(lambda p: color_dict[p.point_type], points_t)))

    n_mens = np.sum(labels == 'C0')
    n_womens = np.sum(labels == 'C1')
    
    if len(x_y) > 0:
        
        fig, ax = plt.subplots()
        
        for color in ['C0', 'C1']:   
            ax.scatter([], [], c=color, label=label_dict[color])
        
        ax.scatter(x_y.T[0], x_y.T[1], c=labels, alpha=0.5)
        plt.grid()
        plt.legend()
        plt.suptitle('Mens: {}; Womens: {}'.format(n_mens, n_womens))
        plt.xlim(xlim)
        plt.ylim(ylim)
        plt.show()

        

In [26]:
def plot_points_interactive(points, T):
    interval_tree = build_interval_tree(points)

    max_t = get_max_t(points) - 0.01
    min_x, min_y, max_x, max_y = get_min_coords(points)

    interact(lambda t: plot_points(t, points, interval_tree, [min_x, max_x], [min_y, max_y]), t=(0, min(max_t, T)))

In [27]:
def run_simulation(mu = np.array([-1, 0.02, 0.02]), 
                   kappa = np.array([-1, 0.1, 0.1]),
                   size = 20,
                   noise_std = 0.01,
                   beta = beta,
                   a = a,
                   init_points=init_points_2d,
                   max_steps=5000000,
                   T=500, plot=True):

    R = init_points(size)
    H = []

    simulate(R, H, T, mu, kappa, beta, a, noise_std, max_steps)

    all_points = np.array(R + H)

    if plot:
        plot_points_interactive(all_points, T)
    
    return all_points
    

In [28]:
def save_simulation(points, file_nm):
    point_types = [p.point_type for p in points]
    x_1 = [p.x[0] for p in points]
    x_2 = [p.x[1] for p in points]
    t_1 = [p.t_1 for p in points]
    t_2 = [p.t_2 for p in points]

    df = pd.DataFrame(data={
        'point_type': point_types,
        'x_1': x_1,
        'x_2': x_2,
        't_1': t_1,
        't_2': t_2
    })

    df.to_csv(file_nm + '.csv', index=False)


def load_simulation(file_nm, T=2000):
    df = pd.read_csv(file_nm + '.csv')

    all_points = df.apply(lambda row: Point(row.point_type,
                                           np.array([row.x_1, row.x_2]),
                                           row.t_1,
                                           row.t_2), axis=1)
    plot_points_interactive(all_points, T)

    return all_points


# Дает потомство только в 0 $(b_0 = b_2) $

In [29]:
def beta(p, k, l, point_tree_dict):
    if p.point_type == 1:
        return 0
    
    if p.point_type == 2 and p.x[0] == 0 and p.x[1] == 0:
        if k == 1:
            if l == 1:
                return 0.5
        if k == 0:
            if l == 2:
                return 0.5      
    else:
        return 0

In [30]:
all_points = run_simulation(
    mu = np.array([-1, 0.1, 1]),
    max_steps=5000000,
    T=2000,
    beta=beta,
    plot=False
)

  1%|▍                               | 65758/5000000 [00:03<04:54, 16742.10it/s]


In [31]:
save_simulation(all_points, 'simulations/branching_only_at_zero')

In [32]:
all_points = load_simulation('simulations/branching_only_at_zero', T=2000)

100%|█████████████████████████████████| 65758/65758 [00:00<00:00, 124574.27it/s]


interactive(children=(FloatSlider(value=136.40945162059356, description='t', max=272.8189032411871), Output())…

# Дает потомство в любой точке $(b_0 = b_2) $

In [33]:
def beta(p, k, l, point_tree_dict):
    if p.point_type == 1:
        return 0
    
    if p.point_type == 2:
        if k == 1:
            if l == 1:
                return 0.5
        if k == 0:
            if l == 2:
                return 0.5      
    else:
        return 0

In [34]:
all_points = run_simulation(
    mu = np.array([-1, 0.1, 1]),
    max_steps=5000000,
    T=2000,
    beta=beta,
    plot=False
)

  3%|▊                              | 136145/5000000 [00:08<04:52, 16617.27it/s]


In [35]:
save_simulation(all_points, 'simulations/branching_everywhere')

In [36]:
all_points = load_simulation('simulations/branching_everywhere', T=2000)

100%|███████████████████████████████| 136145/136145 [00:01<00:00, 116558.58it/s]


interactive(children=(FloatSlider(value=180.5507342715461, description='t', max=361.1014685430922), Output()),…

# Взаимодействие
$$ \beta(k, l, t, x) = \frac{1}{k + l} \cdot (1 - e^{-min(N_1(t, x), N_2(t, x))}), \space \space k + l = 2 $$
<br>
$\beta(k, l, t, x)$ - интенсивность женской частицы, находящейся в момент времени $t$ в точке $x$, породить $k$ мужских и $l$ женских частиц 
<br>
$N_1(t, x)$ - кол-во мужских частиц в момент времени $t$ в точке $x$
<br>
$N_2(t, x)$ - кол-во женских частиц в момент времени $t$ в точке $x$

In [37]:
import math

def beta(p, k, l, point_tree_dict):
    if p.point_type == 1:
        return 0

    x_str = get_x_str(p.x)

    N_1 = N_2 = 0
    if x_str in point_tree_dict:
        N_1 = np.sum(list(map(lambda i: i.data, point_tree_dict[x_str][0].at(p.t_1))))
        N_2 = np.sum(list(map(lambda i: i.data, point_tree_dict[x_str][1].at(p.t_1))))

    c = 1 - math.exp(-np.min([N_1, N_2]))
    res = 0
    
    if p.point_type == 2:
        if k == 1:
            if l == 1:
                res = 0.5
        if k == 0:
            if l == 2:
                res = 0.5     

    return res * c

In [50]:
all_points = run_simulation(
    mu = np.array([-1, 0.1, 0.6]),
    max_steps=5000000,
    T=2000,
    beta=beta,
    plot=False
)

  2%|▌                               | 87695/5000000 [00:05<05:13, 15651.39it/s]


In [51]:
save_simulation(all_points, 'simulations/branching_interactive_06')

In [52]:
load_simulation('simulations/branching_interactive_06')

100%|█████████████████████████████████| 87695/87695 [00:00<00:00, 118538.03it/s]


interactive(children=(FloatSlider(value=154.6838711235727, description='t', max=309.3677422471454), Output()),…

0                                     1.0 13.0 5.0 0.0 0.0
1                                   2.0 14.0 -20.0 0.0 0.0
2                                    1.0 -12.0 8.0 0.0 0.0
3                                   1.0 12.0 -18.0 0.0 0.0
4                                   1.0 -8.0 -10.0 0.0 0.0
                               ...                        
87690    1.0 -2.910794598013407 1.0802782926050416 305....
87691    1.0 -3.915450025759919 1.0955608708818547 306....
87692    1.0 -2.908547689301072 1.098522487371211 307.5...
87693    1.0 -1.896124696440924 1.1155140325560509 308....
87694    1.0 -2.9101404780604563 0.1228252733796843 309...
Length: 87695, dtype: object

In [68]:
all_points = run_simulation(
    mu = np.array([-1, 0.1, 0.1]),
    max_steps=20000000,
    T=2000,
    beta=beta,
    plot=False
)

 34%|█████████▌                  | 6864260/20000000 [26:40<1:26:37, 2527.45it/s]

KeyboardInterrupt: 

In [ ]:
save_simulation(all_points, 'simulations/branching_interactive_01')

In [ ]:
load_simulation('simulations/branching_interactive_01')

 34%|█████████▌                  | 6864484/20000000 [26:55<1:26:37, 2527.45it/s]